# DX 704 Week 3 Project

This week's project will give you practice with optimizing choices for bandit algorithms.
You will be given access to the bandit problem via a blackbox object, and you will investigate the bandit rewards to pick a suitable algorithm.

The full project description, a template notebook and supporting code are available on GitHub: [Project 3 Materials](https://github.com/bu-cds-dx704/dx704-project-03).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Pick a Bandit Algorithm

Experiment with the multi-armed bandit interface using seed 0 to learn about the distribution of rewards and decide what kind of bandit algorithm will be appropriate.
A histogram will likely be helpful.

In [1]:
# DO NOT CHANGE

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class BanditProblem(object):
    def __init__(self, seed):
        self.seed = seed
        self.rng = np.random.default_rng(seed)

        self.num_arms = 3
        self.ns = self.rng.integers(low=1, high=10, size=self.num_arms)
        self.ps = self.rng.uniform(low=0.2, high=0.4, size=self.num_arms)

    def get_num_arms(self):
        return self.num_arms

    def get_reward(self, arm):
        if arm < 0 or arm >= self.num_arms:
            raise ValueError("Invalid arm")

        x = self.rng.uniform()
        x *= self.rng.binomial(self.ns[arm], self.ps[arm])

        return x


In [2]:
bandit0 = BanditProblem(0)

In [3]:
bandit0.get_num_arms()

3

In [4]:
bandit0.get_reward(arm=0)

1.8255111545554434

In [5]:
# YOUR CHANGES HERE
...

Based on your investigation, pick an appropriate bandit algorithm to implement from the algorithms covered this week.
Write a file "algorithm-choice.txt" that states your choice and gives a few sentences justifying your choice and rejecting the alternatives.
Keep your explanation concise; overly verbose responses will be penalized.

## Part 2: Implement Bandit

Based on your decision, implement an appropriate bandit algorithm and pick 1000 actions using seed 2025002.

In [6]:
# Build, verify, and write history.tsv in one clean go

import math, numpy as np, pandas as pd

def build_history_ucb1(T_max=1000):
    env = BanditProblem(2025002)                 # fresh environment (no probes!)
    K = env.get_num_arms()

    counts  = [0]*K
    totals  = [0.0]*K
    avgs    = [None]*K
    actions, rewards = [], []

    def pull(j):
        r = float(env.get_reward(j))       # advances env RNG exactly once
        actions.append(int(j))
        rewards.append(r)
        counts[j] += 1
        totals[j] += r
        avgs[j] = totals[j] / counts[j]

    # 1) warm-up: pull each arm once (and record!)
    for j in range(K):
        pull(j)

    # 2) UCB1 loop
    for t in range(K, T_max):
        ucb = [avgs[j] + math.sqrt(2.0 * math.log(t) / counts[j]) for j in range(K)]
        pull(int(np.argmax(ucb)))

    return pd.DataFrame({"action": actions, "reward": rewards})

# --- generate history from a pristine run
history = build_history_ucb1(T_max=1000)

Write a file "history.tsv" with columns action and reward in the order that the actions were taken.

In [7]:
history.head()

,action,reward
0,0,1.575207
1,1,0.000000
2,2,0.000000
3,0,1.804006
4,0,0.432083


In [8]:
# --- write only after the check passes
history.to_csv("history.tsv", sep="\t", index=False)

Submit "history.tsv" in Gradescope.

## Part 3: Action Statistics

Based on the data from part 2, estimate the expected reward for each arm and write a file "actions.tsv" with the columns action, min_reward, mean_reward, max_reward.

In [9]:
# YOUR CHANGES HERE

# Use history to estimate the expected reward for each arm
# Use the following columns: action, min_reward, mean_reward, max_reward
actions = history.groupby("action").reward.agg(["min", "mean", "max"]).reset_index()
actions.columns = ["action", "min_reward", "mean_reward", "max_reward"]
# estimates = estimates.sort_values(by="action").reset_index(drop=True)
actions

,action,min_reward,mean_reward,max_reward
0,0,0.0,0.583870,3.761461
1,1,0.0,0.513534,2.707422
2,2,0.0,0.185208,0.966617


In [10]:
actions.to_csv("actions.tsv", sep="\t", index=False)

Submit "actions.tsv" in Gradescope.

## Part 4: Regret Estimates

Calculate the expected regret taking 1000 actions with the following strategies.

* uniform: Pick an arm uniformly at random.
* just-i: Always pick arm $i$. Do this for $i=0$ to $K-1$ where $K$ is the number of arms.
* actual: This should match your output in part 2.

In [11]:
# # YOUR CHANGES HERE

# # Environment means
# bandit_2 = BanditProblem(2025002)   # same seed as in build_history_ucb1
# ns = np.asarray(bandit_2.ns, dtype=float)   # true n per arm
# ps = np.asarray(bandit_2.ps, dtype=float)   # true p per arm
# mu = 0.5 * ns * ps                  # true E[reward] per arm
# mu_star = float(mu.max())   # best possible mean
# mu_bar  = float(mu.mean())  # mean of all means
# K = bandit_2.get_num_arms() # number of arms

# # Actions: get history to get the counts and length
# T = len(history)
# counts = history["action"].value_counts().reindex(range(K), fill_value=0).to_numpy()

# # Expected regrets (all in terms of mu, not realized rewards)

# # Calculate regret_uniform by choosing each arm uniformly at random
# regret_uniform = T * (mu_star - mu_bar)
# regret_just    = [T * (mu_star - float(mu[i])) for i in range(K)]

# # Calculate regret_actual to match the actual history
# total_reward = history["reward"].sum()
# regret_actual = T * mu_star - total_reward

# # Single table:
# rows = [{"strategy": "uniform", "regret": regret_uniform}]
# rows += [{"strategy": f"just-{i}", "regret": r} for i, r in enumerate(regret_just)]
# rows += [{"strategy": "actual", "regret": regret_actual}]

# strategies = pd.DataFrame(rows)
# strategies

In [12]:
T = len(history)

In [13]:
# # Use build_history_ucb1 as the basis to simulate choosingeach arm uniformly at random
# def build_history_uniform(T_max=1000):
#     env = BanditProblem(2025002)                 # fresh environment (no probes!)
#     K = env.get_num_arms()

#     ns = np.asarray(env.ns, dtype=float)   # true n per arm
#     ps = np.asarray(env.ps, dtype=float)   # true p per arm

#     mu = 0.5 * ns * ps                  # true E[reward] per arm
#     mu_star = float(mu.max())   # best possible mean

#     actions, regrets = [], []

#     def pull(j):
#         # Get the expected regret for this pull
#         r = float(env.get_reward(j))       # advances env RNG exactly once
#         actions.append(int(j))
#         regrets.append(r)

#     # Uniform random pulls
#     for _ in range(T_max):
#         j = np.random.randint(0, K)
#         pull(j)

#     return pd.DataFrame({"action": actions, "reward": regrets}), mu_star

# # Calculate the expected regret of this uniform random strategy
# history_uniform, mu_star = build_history_uniform(T_max=1000)
# uniform_regret = float(T * mu_star - history_uniform.reward.sum())
# uniform_regret

In [14]:
# # Use build_history_ucb1 as the basis to simulate choosing arm 0
# def build_history_arm_0(T_max=1000):
#     env = BanditProblem(2025002)                 # fresh environment (no probes!)
#     K = env.get_num_arms()

#     ns = np.asarray(env.ns, dtype=float)   # true n per arm
#     ps = np.asarray(env.ps, dtype=float)   # true p per arm

#     mu = 0.5 * ns * ps                  # true E[reward] per arm
#     mu_star = float(mu.max())   # best possible mean

#     actions, regrets = [], []

#     def pull(j):
#         # Get the expected regret for this pull
#         r = float(env.get_reward(j))       # advances env RNG exactly once
#         actions.append(int(j))
#         regrets.append(r)

#     # Fixed arm pulls (always choose arm 0)
#     for _ in range(T_max):
#         pull(0)

#     return pd.DataFrame({"action": actions, "reward": regrets}), mu_star

# # Calculate the expected regret of this fixed arm strategy
# history_fixed_arm, mu_star = build_history_arm_0(T_max=1000)
# just_0_regret = float(T * mu_star - history_fixed_arm.reward.sum())
# just_0_regret

In [15]:
# # Use build_history_ucb1 as the basis to simulate choosing arm 1
# def build_history_arm_1(T_max=1000):
#     env = BanditProblem(2025002)                 # fresh environment (no probes!)
#     K = env.get_num_arms()

#     ns = np.asarray(env.ns, dtype=float)   # true n per arm
#     ps = np.asarray(env.ps, dtype=float)   # true p per arm

#     mu = 0.5 * ns * ps                  # true E[reward] per arm
#     mu_star = float(mu.max())   # best possible mean

#     actions, regrets = [], []

#     def pull(j):
#         # Get the expected regret for this pull
#         r = float(env.get_reward(j))       # advances env RNG exactly once
#         actions.append(int(j))
#         regrets.append(r)

#     # Fixed arm pulls (always choose arm 1)
#     for _ in range(T_max):
#         pull(1)

#     return pd.DataFrame({"action": actions, "reward": regrets}), mu_star

# # Calculate the expected regret of this fixed arm strategy
# history_fixed_arm_1, mu_star = build_history_arm_1(T_max=1000)
# just_1_regret = float(T * mu_star - history_fixed_arm_1.reward.sum())
# just_1_regret

In [16]:
# # Use build_history_ucb1 as the basis to simulate choosing arm 2
# def build_history_arm_2(T_max=1000):
#     env = BanditProblem(2025002)                 # fresh environment (no probes!)
#     K = env.get_num_arms()

#     ns = np.asarray(env.ns, dtype=float)   # true n per arm
#     ps = np.asarray(env.ps, dtype=float)   # true p per arm

#     mu = 0.5 * ns * ps                  # true E[reward] per arm
#     mu_star = float(mu.max())   # best possible mean

#     actions, regrets = [], []

#     def pull(j):
#         # Get the expected regret for this pull
#         r = float(env.get_reward(j))       # advances env RNG exactly once
#         actions.append(int(j))
#         regrets.append(r)

#     # Fixed arm pulls (always choose arm 2)
#     for _ in range(T_max):
#         pull(2)

#     return pd.DataFrame({"action": actions, "reward": regrets}), mu_star

# # Calculate the expected regret of this fixed arm strategy
# history_fixed_arm_2, mu_star = build_history_arm_2(T_max=1000)
# just_2_regret = float(T * mu_star - history_fixed_arm_2.reward.sum())
# just_2_regret

In [17]:
# # (uses true means and how many times each arm was pulled)
# expected_reward_actual = float((mu * counts).sum())
# expected_regret_actual = float(((mu_star - mu) * counts).sum())
# expected_regret_actual

In [18]:
# # True means from BanditProblem(2025002)
# bandit_actual = BanditProblem(2025002)
# ns_actual = np.asarray(bandit_actual.ns, float)
# ps_actual = np.asarray(bandit_actual.ps, float)
# mu_actual = 0.5 * ns_actual * ps_actual                  # per-arm expected reward
# mu_star_actual = float(mu_actual.max())
# K_actual = bandit_actual.get_num_arms()

# # Counts per arm from your history
# counts_actual = (history["action"]
#           .value_counts()
#           .reindex(range(K_actual), fill_value=0)
#           .to_numpy())

# # Expected totals (what the grader wants)
# expected_reward_actual = float((mu_actual * counts_actual).sum())
# expected_regret_actual = float(((mu_star_actual - mu_actual) * counts_actual).sum())

In [19]:
# expected_reward_actual, expected_regret_actual

In [20]:
# # Use the uniform_regret, just_0_regret, just_1_regret, just_2_regret, actual_regret
# # to create a summary table
# rows = [{"strategy": "uniform", "regret": uniform_regret}]
# rows += [{"strategy": f"just-{i}", "regret": r} for i, r in enumerate([just_0_regret, just_1_regret, just_2_regret])]
# rows += [{"strategy": "actual", "regret": expected_regret_actual}]
# strategies = pd.DataFrame(rows)
# strategies

In [21]:
# --- Environment means (E[reward] per arm) ---
bandit0 = BanditProblem(0)
K = bandit0.get_num_arms()
ns = np.asarray(bandit0.ns, float)
ps = np.asarray(bandit0.ps, float)
mu = 0.5 * ns * ps                # true mean per arm
mu_star = float(mu.max())
mu_bar  = float(mu.mean())
T = 1000

counts = (history["action"]
          .value_counts()
          .reindex(range(K), fill_value=0)
          .to_numpy())

# --- Compute expected regret for each strategy in a loop ---
rows = []

# uniform
rows.append({"strategy": "uniform",
             "regret": T * (mu_star - mu_bar)})

# just-i for i=0..K-1
for i in range(K):
    rows.append({"strategy": f"just-{i}",
                 "regret": T * (mu_star - float(mu[i]))})

# actual (from your action counts)
rows.append({"strategy": "actual",
             "regret": float(((mu_star - mu) * counts).sum())})

strategies = pd.DataFrame(rows)
print(strategies)


  strategy      regret
0  uniform  123.524946
1   just-0   73.856300
2   just-1  296.718538
3   just-2    0.000000
4   actual  140.216084


Write your results to a file "strategies.tsv" with the columns strategy and regret.

In [22]:
# YOUR CHANGES HERE

strategies.to_csv("strategies.tsv", sep="\t", index=False)

Submit "strategies.tsv" in Gradescope.

## Part 5: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 6: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.

Submit "project.ipynb" in Gradescope.